# RAG

## Requirements

In [1]:
%%capture
!pip install transformers accelerate bitsandbytes langchain langchain-community sentence-transformers faiss-gpu pandas gdown

## Dataset

In [2]:
!gdown --fuzzy https://drive.google.com/file/d/1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI/view?usp=sharing

Downloading...
From (original): https://drive.google.com/uc?id=1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI
From (redirected): https://drive.google.com/uc?id=1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI&confirm=t&uuid=ea41d9a3-28c8-4269-958e-264e987b7278
To: /content/IMDB_crawled.json
100% 292M/292M [00:02<00:00, 134MB/s]


## Config

In [3]:
class Config:
    EMBEDDING_MODEL_NAME="thenlper/gte-base"
    LLM_MODEL_NAME="HuggingFaceH4/zephyr-7b-beta"
    K = 5 # top K retrieval

## Preprocessing

In [4]:
import pandas as pd

df = pd.read_json('IMDB_crawled.json')

In [5]:
import os

os.makedirs('data', exist_ok=True)

# preprocess your data and only store the needed data as the context window for embedding model is limited
def preprocess_data(df):
    df = df[['title', 'first_page_summary', 'summaries', 'genres']]
    df = df.dropna()
    df['summaries'] = df['summaries'].apply(lambda x: x[0])
    df = df.astype(str)

    df = df.fillna('')
    df['combined_text'] = 'title: ' + df['title'] + ' first_page_summaries: ' + \
     df['first_page_summary'] + ' summaries: ' + df['summaries'][0] + ' genres: ' + df['genres']
    df = df[['combined_text']]

    return df

df = preprocess_data(df)

df.to_csv('data/imdb.csv', index=False)

## Vectorizer

load the CSV file and vectorize the rows using HuggingFaceEmbeddings.
Store the results using FAISS vectorstore.
Save the vectorestore in a pickle file for future usages.

In [6]:
import pickle

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores.utils import DistanceStrategy
from langchain.vectorstores.faiss import FAISS

from langchain_community.embeddings import HuggingFaceEmbeddings

In [7]:
# load the csv
loader = CSVLoader(file_path='data/imdb.csv')
documents = loader.load()

# load the embeddings model
embeddings_model = HuggingFaceEmbeddings(model_name=Config.EMBEDDING_MODEL_NAME)

# save embed the documents using the model in a vectorstore
vectorstore = FAISS.from_documents(documents, embeddings_model)

with open("data/vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

load the vectorstore as a retriever.

In [8]:
with open("data/vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

# load the retriever from the vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": Config.K})

## LLM

load the quantized LLM.

In [9]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# load the quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True,
)


model = AutoModelForCausalLM.from_pretrained(Config.LLM_MODEL_NAME, quantization_config=bnb_config, device_map="cuda:0", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(Config.LLM_MODEL_NAME)

# init the pipeline
READER_LLM = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64,
    torch_dtype=torch.float16,
)

llm = HuggingFacePipeline(
    pipeline=READER_LLM,
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


initialize the prompt template for the query chain. query chain is used to get a query from the chat history. you may change the prompt as you like to get better results.

In [10]:
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain

class LoggerStrOutputParser(StrOutputParser):
    def parse(self, text: str) -> str:
        # process the LLM output
        print(f"QUERY: {text}")
        return text

query_transform_prompt = PromptTemplate(
    input_variables=["messages"],
    template="""<|system|>You are a helpful assistant.
{messages}
<|user|>
give me the search query about the above conversation.
<|assistant|>"""
)

# init the query chain
class LoggerStrOutputParser(StrOutputParser):
    def parse(self, text: str) -> str:
        # process the LLM output
        print(f"QUERY: {text}")
        return text

query_transforming_retriever_chain = LLMChain(
    llm=llm,
    prompt=query_transform_prompt,
    output_parser=LoggerStrOutputParser(),
    verbose=True
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


initialize the main retrieval chain that gives the resulting documents to LLM and gets the output back.

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate(
    input_variables=["context", "messages"],
    template="""<|system|>You are a helpful assistant.

Here are the movies you MUST choose from:

{context}
-----------------
{messages}
<|assistant|>""")

# init the retriver chain
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = (
    {"context": retriever, "messages": RunnablePassthrough()}
    | document_chain
    | StrOutputParser()
)

write the conversation helper class for easier testing.

In [12]:
class Conversation:
    def __init__(self):
        self.messages = []

    def add_assistant_message(self, message):
        self.messages.append(('assistant', message))

    def add_user_message(self, message):
        self.messages.append(('user', message))

    def get_messages(self):
        # concatenate the messages with the roles in the instruction format
        formatted_messages = []
        for role, content in self.messages:
            formatted_messages.append(f"<|{role}|>\n{content}")
        return "\n".join(formatted_messages)

    def chat(self, message):
        self.add_user_message(message)
        messages = self.get_messages()
        # invoke the chain
        query = query_transforming_retriever_chain.run({"messages": messages})
        response = retrieval_chain.invoke(query)
        self.add_assistant_message(response)
        return response

## Test

talk with the RAG to see how good it performs.

In [13]:
c = Conversation()
A = c.chat('give me a cool gangster movie')
print(A)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
<|system|>You are a helpful assistant.
<|user|>
give me a cool gangster movie
<|user|>
give me the search query about the above conversation.
<|assistant|>
QUERY: <|system|>You are a helpful assistant.
<|user|>
give me a cool gangster movie
<|user|>
give me the search query about the above conversation.
<|assistant|>
"recommend a cool gangster movie" or "suggest a gripping gangster film" or "what's a must-watch gangster movie?" or "what's the best gangster movie to watch right now?" or "what's a popular gangster movie that's worth

> Finished chain.
<|system|>You are a helpful assistant.

Here are the movies you MUST choose from:

combined_text: title: American Gangster first_page_summaries: Follows the lives of American gangsters. summaries: The early life and career of Vito Corleone in 1920s New York City is portrayed, while his son, Michael, expands and tightens his grip on the family crime syndicate. genres: ['Documentary'

In [14]:
A = c.chat('give me a newer one')
print(A)



> Entering new LLMChain chain...
Prompt after formatting:
<|system|>You are a helpful assistant.
<|user|>
give me a cool gangster movie
<|assistant|>
<|system|>You are a helpful assistant.

Here are the movies you MUST choose from:

combined_text: title: American Gangster first_page_summaries: Follows the lives of American gangsters. summaries: The early life and career of Vito Corleone in 1920s New York City is portrayed, while his son, Michael, expands and tightens his grip on the family crime syndicate. genres: ['Documentary']

combined_text: title: American Gangster first_page_summaries: An outcast New York City cop is charged with bringing down Harlem drug lord Frank Lucas, whose real life inspired this partly biographical film. summaries: The early life and career of Vito Corleone in 1920s New York City is portrayed, while his son, Michael, expands and tightens his grip on the family crime syndicate. genres: ['Biography', 'Crime', 'Drama']

combined_text: title: Jigarthanda fir